# Retrieve data from Instagram using Facebook Graph API

In [9]:
# import library 
import requests
import pandas as pd

# Set access token
access_token = 'Fill your access token'

# Make API request
url_id = 'https://graph.facebook.com/v13.0/me/accounts'
params_id = {'fields': 'created_time,message', 'access_token': access_token}
response_id = requests.get(url_id, params=params_id)

# Convert response to dataframe
data_id = response_id.json()
df_id = pd.json_normalize(data_id['data'])


# Make API request to get account id 
url_acc = f'https://graph.facebook.com/v13.0/{df_id["id"][0]}?fields=connected_instagram_account&access_token={access_token}'
response_acc = requests.get(url_acc)
data_acc = response_acc.json()
df_acc = pd.json_normalize(data_acc)


# Make API request to get media id or post id 
# Set Instagram account ID
account_id = df_acc["connected_instagram_account.id"][0]

# Set API endpoint and parameters
url_media = f'https://graph.facebook.com/v13.0/{account_id}/media'
params_media = {
    'metric': 'id',
    'access_token': access_token
}

# Make API request
response_media = requests.get(url_media, params=params_media)

data_media = response_media.json()
media_data = data_media['data']
next_page = data_media['paging'].get('next')

while next_page:
    response_media = requests.get(next_page)
    data_media = response_media.json()
    media_data += data_media['data']
    next_page = data_media['paging'].get('next')


id_media = []
for i in range(len(media_data)):
    id_media.append(media_data[i]["id"])
    
    

# Looping to get data 
df_on_assets = []
for i in range(len(id_media)):

    url_ass = f'https://graph.facebook.com/v13.0/{id_media[i]}'
    params_ass = {
    'fields': 'username,like_count,comments_count,media_product_type,media_type,timestamp,permalink',
    'access_token': access_token}
    response_ass = requests.get(url_ass, params=params_ass)
    data_ass = response_ass.json()
    df_ass = pd.json_normalize(data_ass)
    
    df_on_assets.append(data_ass)
df_ass = pd.DataFrame(df_ass)


reach_d = []
imp_d = []
save_d = []
eng_d = []

data_nol = 0

for i in range(len(id_media)):
    # Set API endpoint and parameters
    url = f'https://graph.facebook.com/v13.0/{id_media[i]}'
    params = {
        'fields': 'media_product_type',
        'access_token': access_token
    }

    # Make API request to get media type
    response = requests.get(url, params=params)

    # Check media type and either continue loop or update required metrics with zeros
    data = response.json()
    tipe = data.get('media_product_type')
    
    if tipe == "REELS":
        try:
            url = f'https://graph.facebook.com/v13.0/{id_media[i]}/insights'
            params = {
            'metric': 'saved,reach',
            'access_token': access_token}

            # Make API request
            response = requests.get(url, params=params)

            # Convert response to dataframe
            data = response.json()
            df = pd.json_normalize(data['data'])
            save_d.append(df["values"][0][0]["value"])
            reach_d.append(df["values"][1][0]["value"])
            imp_d.append(data_nol)
            eng_d.append(data_nol)
        except requests.exceptions.HTTPError as e:
            #print(f"Error occurred while retrieving media data for media_id {id_media}: {e}")
            continue
        except Exception as e:
            #print(f"Unknown error occurred while retrieving media data for media_id {id_media}: {e}")
            continue
    else:
        try:
            url = f'https://graph.facebook.com/v13.0/{id_media[i]}/insights'
            params = {
            'metric': 'saved,reach,impressions,engagement',
            'access_token': access_token}

            # Make API request
            response = requests.get(url, params=params)

            # Convert response to dataframe
            data = response.json()
            df = pd.json_normalize(data['data'])
            save_d.append(df["values"][0][0]["value"])
            reach_d.append(df["values"][1][0]["value"])
            imp_d.append(df["values"][2][0]["value"])
            eng_d.append(df["values"][3][0]["value"])
        except requests.exceptions.HTTPError as e:
            #print(f"Error occurred while retrieving media data for media_id {id_media}: {e}")
            continue
        except Exception as e:
            #print(f"Unknown error occurred while retrieving media data for media_id {id_media}: {e}")
            continue


data_insight = {
    "saved" : save_d,
    "reach" : reach_d,
    "impressions" : imp_d,
    "engagement" : eng_d
}


df_insight = pd.DataFrame(data=data_insight, columns=["saved","reach","impressions","engagement"])
df_insight.head()

df_all = pd.DataFrame(df_on_assets)
df_all.head()

df_final = pd.concat([df_all,df_insight], axis=1)

df_final.fillna(0, inplace=True)

df_final


,username,like_count,comments_count,media_product_type,media_type,timestamp,permalink,id,saved,reach,impressions,engagement
0,rumahbalontgl,39,0,REELS,VIDEO,2022-10-15T06:40:48+0000,https://www.instagram.com/reel/CjuYA2ID-kV/,17855765891830346,2.0,1209.0,0.0,0.0
1,rumahbalontgl,15,1,FEED,IMAGE,2022-10-09T09:52:15+0000,https://www.instagram.com/p/CjfRV0VrTdM/,17907653552645222,1.0,828.0,931.0,17.0
2,rumahbalontgl,11,2,FEED,IMAGE,2022-02-05T07:05:41+0000,https://www.instagram.com/p/CZliukYFknm/,17965611553527458,5.0,1008.0,1139.0,18.0
3,rumahbalontgl,10,0,FEED,IMAGE,2022-02-01T03:37:54+0000,https://www.instagram.com/p/CZa3xS8lR29/,17916298451325500,10.0,1134.0,1307.0,20.0
4,rumahbalontgl,9,0,FEED,IMAGE,2022-01-30T04:26:18+0000,https://www.instagram.com/p/CZVzt_UlHn6/,18220669858114219,6.0,1251.0,1399.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...
164,rumahbalontgl,4,0,FEED,IMAGE,2015-05-19T08:20:49+0000,https://www.instagram.com/p/220lq_Ia2z/,17842117036070155,0.0,0.0,0.0,0.0
165,rumahbalontgl,4,0,FEED,IMAGE,2015-05-19T08:20:03+0000,https://www.instagram.com/p/220f_7Ia2v/,17842117033070155,0.0,0.0,0.0,0.0
166,rumahbalontgl,6,0,FEED,IMAGE,2015-05-19T08:19:12+0000,https://www.instagram.com/p/220ZzJIa2m/,17842117027070155,0.0,0.0,0.0,0.0
167,rumahbalontgl,7,0,FEED,IMAGE,2015-05-19T08:18:34+0000,https://www.instagram.com/p/220VPKoa2j/,17842117024070155,0.0,0.0,0.0,0.0


In [14]:
df_final.to_excel("data_contoh.xlsx")